Pre-installed library imports

In [2]:

import importlib
import subprocess
import sys

Function to check if the libraries used in this code are already installed

In [3]:
def checkIfInstalled(library):
    try:
        importlib.import_module(library)
        print(f'{library} is installed')
    
    except ImportError:
        print(f'{library} is not installed, installing...')
        
        # Using sys.executable to install the library in the same python version, and -m to run the module as a script and ensure the correct version is used
        subprocess.run([sys.executable,'-m', 'pip', 'install', library])

Aditional library imports 

In [4]:
# Requests library, used to make HTTP requests
checkIfInstalled('requests')
import requests

requests is installed


TBA API setup

In [5]:
url = "https://www.thebluealliance.com/api/v3"
header = {"X-TBA-Auth-Key":"E5xeD6sWIPJe5QDAgtoXzoHkQpKxaoOJlyGXThQBH4SztWmnfEgSbjYLFbC40r1E"}

# Availability check
testResponse = requests.get(url + "/status", headers=header)

print(testResponse.status_code)

200


Fetch individual team data 
    -fetchRecursive: Recursively iterates through a list of dictionaries containing team data checking coincidences on a given team number, if there's a match, a dictionary containing team data is returned, else, an empty list is returned

    -fetchTeamData: Recursively extracts team lists from TBA API and uses fetchRecursive to find a team's data given a team number

FRC team class 

In [17]:
class Team:
    # Static Methods
    @staticmethod
    def fetchRecursive(teamsJson, teamNumber):
        if len(teamsJson) == 0:
            return []
        else:
            if (teamNumber == teamsJson[0]["team_number"]):
                return teamsJson[0] # Returns team dictionary
            else:
                return Team.fetchRecursive(teamsJson[1:], teamNumber)
               
    @staticmethod
    def fetchTeamData(teamNumber, page = 0):
        teamsJson = requests.get(url + f"/teams/2023/{page}/simple", headers=header).json()
    
        if len(teamsJson) == 0:
            return []
        else:
            out = Team.fetchRecursive(teamsJson, teamNumber)
        if type(out) == dict:
            return out 
        else:
            return Team.fetchTeamData(teamNumber, page + 1)
        
    # Class Methods    
    @classmethod
    def instanceByTeamNumber(cls, teamNumber):
        return Team(Team.fetchTeamData(teamNumber))

    # Constructor
    def __init__(self, teamDict):
        # Private atrributes
        self.__teamKey = teamDict["key"]
        
        # Public attributes
        self.teamNumber = teamDict["team_number"]
        self.name = teamDict["name"]
        self.city = teamDict["city"]
        self.state = teamDict["state_prov"]
        self.country = teamDict["country"]
        
    @property
    def teamKey(self):
        return self.__teamKey
    
    def __repr__(self):
        return f"Team {self.teamNumber}"

team4403 = Team.instanceByTeamNumber(4403)

print(team4403.__dict__)

{'_Team__teamKey': 'frc4403', 'teamNumber': 4403, 'name': 'PEÑOLES&PrepaTec Laguna', 'city': 'Torreon', 'state': 'Coahuila', 'country': 'Mexico'}
